In [23]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import psycopg2
import pickle

Questions for Data Preparation
Unsupervised learning doesn't have a clear outcome or target variable like supervised learning, but it is used to find patterns. By properly preparing the data, we can select features that help us find patterns or groups.

Before we begin, consider these questions:

What knowledge do we hope to glean from running an unsupervised learning model on this dataset?
What data is available? What type? What is missing? What can be removed?
Is the data in a format that can be passed into an unsupervised learning model?
Can I quickly hand off this data for others to use?
Let's address the first question on our list:

What knowledge do we hope to glean from running an unsupervised learning model on this dataset?


# Setting up dataframe for unsupervised learning test

In [24]:
ml_df = pd.read_csv("../Resources/full_df_non_nan.csv")

In [25]:
ml_df

,MLSNumber,Orig_List_Price,Zip_Code,Subdivision,School_District,HOA_YN,Structure_Type,Acres,Age,InteriorSqFt,...,Water,Sewer,Basement_YN,Garage_YN,ExteriorMaterial,LotFrontage,LotDepth,LotSqFt,LotAcres,YearBuilt
0,DENC518086,330000.0,19808,SHERWOOD PARK II,Red Clay Consolidated,Yes,Detached,0.26,1958.0,2099.0,...,Public,PublicSewer,Yes,Yes,"BrickFront,VinylSiding",72.4,133.5,11326,0.26,1958
1,DENC518982,215500.0,19808,PEPPER RIDGE,Red Clay Consolidated,No,Interior Row/Townhouse,0.06,1976.0,1927.0,...,Public,PublicSewer,Yes,No,"AluminumSiding,Brick,VinylSiding",16.2,126.2,2614,0.06,1976
2,DENC512992,200000.0,19808,WOODMILL,Red Clay Consolidated,No,Interior Row/Townhouse,0.05,1985.0,1150.0,...,Public,PublicSeptic,Yes,No,VinylSiding,20.0,105.0,2178,0.05,1985
3,DENC512104,219900.0,19808,FAIRWAY FALLS,Red Clay Consolidated,Yes,Interior Row/Townhouse,0.05,1985.0,1425.0,...,Public,PublicSewer,Yes,Yes,Other,16.0,125.0,2178,0.05,1985
4,DENC506356,195000.0,19808,WASHINGTON HEIGHTS,Red Clay Consolidated,No,Detached,0.14,1954.0,0.0,...,Public,PublicSewer,Yes,Yes,"Concrete,Stucco",50.0,118.0,6098,0.14,1954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,DENC520114,595000.0,19707,CANTERBURY HILLS,Red Clay Consolidated,Yes,Detached,1.03,1958.0,3045.0,...,Public,PublicSewer,Yes,Yes,"Stone,VinylSiding",0.0,0.0,44866,1.03,1958
4914,DENC520392,639000.0,19707,SNUFF MILL,Red Clay Consolidated,Yes,Detached,0.75,1973.0,3811.0,...,Well,OnSiteSeptic,Yes,Yes,"Cedar,Combination,Mixed,MixedPlumbing",20.0,300.0,32670,0.75,1973
4915,DENC2014038,529900.0,19707,STENNING WOODS,Red Clay Consolidated,No,Detached,0.39,1989.0,3191.0,...,Public,PublicSewer,Yes,Yes,"AluminumSiding,Brick,VinylSiding",113.3,145.0,16988,0.39,1989
4916,DENC2005484,620000.0,19707,QUAKER LEA EAST,Red Clay Consolidated,No,Detached,0.54,1992.0,3425.0,...,Public,PublicSewer,Yes,Yes,Brick,125.0,190.0,23522,0.54,1992


In [26]:
ml_df.drop(columns="Age" , inplace=True)

In [27]:
ml_df.nunique()

MLSNumber           4918
Orig_List_Price      576
Zip_Code              13
Subdivision          629
School_District        4
HOA_YN                 2
Structure_Type         8
Acres                170
InteriorSqFt        1040
AboveGradeSqFt       406
BelowGradeSqFt       595
Bedrooms               9
Baths                  7
BathsFull              7
Design                10
Style                119
NumberofStories       35
Central_Air_YN         2
Cooling               80
PrimaryHeat           38
HeatDelivery         132
HotWater              25
Water                 15
Sewer                 37
Basement_YN            2
Garage_YN              2
ExteriorMaterial     293
LotFrontage         1052
LotDepth            1192
LotSqFt              196
LotAcres             167
YearBuilt            151
dtype: int64

In [28]:
from config import db_password

In [29]:
# Name the SQL database your are accessing
database = "RealLeads"

In [30]:
# Make your local connection to the database in PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database=database,
    user="postgres",
    password=db_password)

In [31]:
# Table selection Function
def table_select(table_name):
    table =f"select * from \"{table_name}\""
    dataFrame = pd.read_sql(table, conn)
    return dataFrame

In [32]:
sales_data_df = table_select("sales_data_clean")

In [33]:
sales_data_df

,MLSNumber,Address,Status,Sold_Price,Sold_Price_less_Concession,Orig_List_Price,Current_List_Price,Days_on_Market,Previous_Days_on_Market,ListDate,StatusDate,Agreement_of_Sale_Date,SettledDate,Concessions_YN,Concessions_Remarks,SellerConcessionsAmount,FinalFinancing
0,DENC518086,2615 Pecksniff Rd,Closed,335000.0,335000.0,330000.0,330000.0,5.0,5.0,2020-12-11,2021-02-11,2020-12-16,2021-02-11,No,None,0.0,FHA
1,DENC518982,4938 S Tupelo Turn,Closed,200000.0,200000.0,215500.0,210000.0,47.0,67.0,2021-01-07,2021-04-08,2021-02-21,2021-04-08,No,None,0.0,Conventional
2,DENC512992,15 Kristina Ct,Closed,200000.0,200000.0,200000.0,200000.0,15.0,15.0,2020-12-28,2021-02-26,2021-01-09,2021-02-26,No,None,0.0,Conventional
3,DENC512104,3251 Champions Dr,Closed,200000.0,200000.0,219900.0,214900.0,11.0,11.0,2020-10-27,2020-12-14,2020-11-05,2020-12-11,No,None,0.0,Conventional
4,DENC503480,3706 Lafayette St,Closed,200000.0,200000.0,190000.0,190000.0,3.0,3.0,2020-06-20,2020-08-02,2020-06-21,2020-07-31,No,None,0.0,Conventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,DENC520114,101 Meriden Dr,Closed,595000.0,595000.0,595000.0,595000.0,5.0,5.0,2021-06-24,2021-07-28,2021-06-28,2021-07-28,No,None,0.0,Cash
5572,DENC520392,417 Snuff Mill Hill Rd,Closed,595000.0,591262.0,639000.0,639000.0,25.0,25.0,2021-03-29,2021-06-10,2021-04-21,2021-06-10,Yes,None,3738.0,Conventional
5573,DENC2014038,400 Wilson Ct,Closed,600000.0,600000.0,529900.0,529900.0,7.0,7.0,2022-01-14,2022-03-04,2022-01-19,2022-03-04,No,None,0.0,Conventional
5574,DENC2005484,918 Old Public Rd,Closed,600000.0,598000.0,620000.0,620000.0,25.0,25.0,2021-08-26,2021-10-27,2021-09-18,2021-10-26,Yes,Inspections credit,2000.0,Conventional


In [34]:
ml_df = pd.merge(ml_df,sales_data_df[["MLSNumber", "Sold_Price", "Days_on_Market"]] , on="MLSNumber")

In [35]:
ml_df

,MLSNumber,Orig_List_Price,Zip_Code,Subdivision,School_District,HOA_YN,Structure_Type,Acres,InteriorSqFt,AboveGradeSqFt,...,Basement_YN,Garage_YN,ExteriorMaterial,LotFrontage,LotDepth,LotSqFt,LotAcres,YearBuilt,Sold_Price,Days_on_Market
0,DENC518086,330000.0,19808,SHERWOOD PARK II,Red Clay Consolidated,Yes,Detached,0.26,2099.0,2099.0,...,Yes,Yes,"BrickFront,VinylSiding",72.4,133.5,11326,0.26,1958,335000.0,5.0
1,DENC518982,215500.0,19808,PEPPER RIDGE,Red Clay Consolidated,No,Interior Row/Townhouse,0.06,1927.0,1575.0,...,Yes,No,"AluminumSiding,Brick,VinylSiding",16.2,126.2,2614,0.06,1976,200000.0,47.0
2,DENC512992,200000.0,19808,WOODMILL,Red Clay Consolidated,No,Interior Row/Townhouse,0.05,1150.0,1150.0,...,Yes,No,VinylSiding,20.0,105.0,2178,0.05,1985,200000.0,15.0
3,DENC512104,219900.0,19808,FAIRWAY FALLS,Red Clay Consolidated,Yes,Interior Row/Townhouse,0.05,1425.0,1425.0,...,Yes,Yes,Other,16.0,125.0,2178,0.05,1985,200000.0,11.0
4,DENC506356,195000.0,19808,WASHINGTON HEIGHTS,Red Clay Consolidated,No,Detached,0.14,0.0,0.0,...,Yes,Yes,"Concrete,Stucco",50.0,118.0,6098,0.14,1954,201000.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,DENC520114,595000.0,19707,CANTERBURY HILLS,Red Clay Consolidated,Yes,Detached,1.03,3045.0,3045.0,...,Yes,Yes,"Stone,VinylSiding",0.0,0.0,44866,1.03,1958,595000.0,5.0
4914,DENC520392,639000.0,19707,SNUFF MILL,Red Clay Consolidated,Yes,Detached,0.75,3811.0,3175.0,...,Yes,Yes,"Cedar,Combination,Mixed,MixedPlumbing",20.0,300.0,32670,0.75,1973,595000.0,25.0
4915,DENC2014038,529900.0,19707,STENNING WOODS,Red Clay Consolidated,No,Detached,0.39,3191.0,2850.0,...,Yes,Yes,"AluminumSiding,Brick,VinylSiding",113.3,145.0,16988,0.39,1989,600000.0,7.0
4916,DENC2005484,620000.0,19707,QUAKER LEA EAST,Red Clay Consolidated,No,Detached,0.54,3425.0,3425.0,...,Yes,Yes,Brick,125.0,190.0,23522,0.54,1992,600000.0,25.0


In [38]:
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4918 entries, 0 to 4917
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MLSNumber         4918 non-null   object 
 1   Orig_List_Price   4918 non-null   float64
 2   Zip_Code          4918 non-null   int64  
 3   Subdivision       4918 non-null   object 
 4   School_District   4918 non-null   object 
 5   HOA_YN            4918 non-null   object 
 6   Structure_Type    4918 non-null   object 
 7   Acres             4918 non-null   float64
 8   InteriorSqFt      4918 non-null   float64
 9   AboveGradeSqFt    4918 non-null   float64
 10  BelowGradeSqFt    4918 non-null   float64
 11  Bedrooms          4918 non-null   float64
 12  Baths             4918 non-null   float64
 13  BathsFull         4918 non-null   float64
 14  Design            4918 non-null   object 
 15  Style             4918 non-null   object 
 16  NumberofStories   4918 non-null   object 


In [39]:
ml_df.columns

Index(['MLSNumber', 'Orig_List_Price', 'Zip_Code', 'Subdivision',
       'School_District', 'HOA_YN', 'Structure_Type', 'Acres', 'InteriorSqFt',
       'AboveGradeSqFt', 'BelowGradeSqFt', 'Bedrooms', 'Baths', 'BathsFull',
       'Design', 'Style', 'NumberofStories', 'Central_Air_YN', 'Cooling',
       'PrimaryHeat', 'HeatDelivery', 'HotWater', 'Water', 'Sewer',
       'Basement_YN', 'Garage_YN', 'ExteriorMaterial', 'LotFrontage',
       'LotDepth', 'LotSqFt', 'LotAcres', 'YearBuilt', 'Sold_Price',
       'Days_on_Market'],
      dtype='object')

## After establishing a dataframe for unsupervised learning, the next step is to create dummies instead of doing categories for this test.

In [47]:
ml_df_dummies = pd.get_dummies(ml_df,columns=["Zip_Code", "Subdivision","School_District", 'HOA_YN', 'Structure_Type','NumberofStories', 'Central_Air_YN', 'Cooling',
       'PrimaryHeat', 'HeatDelivery', 'HotWater', 'Water', 'Sewer',
       'Basement_YN', 'Garage_YN', 'ExteriorMaterial','Design','Style'])

In [48]:
ml_df_dummies

,MLSNumber,Orig_List_Price,Acres,InteriorSqFt,AboveGradeSqFt,BelowGradeSqFt,Bedrooms,Baths,BathsFull,LotFrontage,...,Style_StraightThru,"Style_StraightThru,Traditional","Style_StraightThru,Transitional",Style_Traditional,"Style_Traditional,Transitional","Style_Traditional,Tudor","Style_Traditional,Victorian",Style_Transitional,Style_Tudor,Style_Victorian
0,DENC518086,330000.0,0.26,2099.0,2099.0,0.0,4.0,2.0,1.0,72.4,...,0,0,0,0,0,0,0,0,0,0
1,DENC518982,215500.0,0.06,1927.0,1575.0,352.0,3.0,2.0,1.0,16.2,...,0,0,0,0,0,0,0,0,0,0
2,DENC512992,200000.0,0.05,1150.0,1150.0,0.0,2.0,2.0,1.0,20.0,...,0,0,0,0,0,0,0,0,0,0
3,DENC512104,219900.0,0.05,1425.0,1425.0,0.0,2.0,3.0,2.0,16.0,...,0,0,0,0,0,0,0,0,0,0
4,DENC506356,195000.0,0.14,0.0,0.0,0.0,4.0,3.0,2.0,50.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,DENC520114,595000.0,1.03,3045.0,3045.0,0.0,3.0,2.0,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4914,DENC520392,639000.0,0.75,3811.0,3175.0,636.0,4.0,4.0,3.0,20.0,...,0,0,0,0,0,0,0,0,0,0
4915,DENC2014038,529900.0,0.39,3191.0,2850.0,341.0,4.0,3.0,2.0,113.3,...,0,0,0,0,0,0,0,0,0,0
4916,DENC2005484,620000.0,0.54,3425.0,3425.0,0.0,4.0,3.0,2.0,125.0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
ml_df_dummies.drop(columns="MLSNumber",inplace=True)

### Scaling data so there is no skewness

In [51]:
scale = StandardScaler()
ml_scaled_dummies = scale.fit_transform(ml_df_dummies)
ml_scaled_dummies[0:5]

array([[ 0.0626134 , -0.01687185,  0.06512974, ..., -0.0285307 ,
        -0.0494569 , -0.08466242],
       [-0.95064332, -0.01899464, -0.13861426, ..., -0.0285307 ,
        -0.0494569 , -0.08466242],
       [-1.08780908, -0.01910077, -1.05901592, ..., -0.0285307 ,
        -0.0494569 , -0.08466242],
       [-0.91170595, -0.01910077, -0.73326244, ..., -0.0285307 ,
        -0.0494569 , -0.08466242],
       [-1.1320561 , -0.01814552, -2.42125777, ..., -0.0285307 ,
        -0.0494569 , -0.08466242]])

## Create a three-dimensional data table using PCA

In [52]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
ml_pca = pca.fit_transform(ml_scaled_dummies)

In [53]:
ml_pca

array([[ 1.28184442,  0.43714553,  0.14414569],
       [-2.21241105,  1.47224915, -1.24829166],
       [-2.55509238,  1.40944799, -1.98767031],
       ...,
       [ 4.36001733,  1.26470464, -0.21021088],
       [ 4.90723879,  1.15602848,  0.47372178],
       [-0.30047765,  1.53472826, -2.37309015]])

In [54]:
# Create a DataFrame with the three principal components.
df_ml_pca = pd.DataFrame(data=ml_pca, columns=["PC1", "PC2", "PC3"])
df_ml_pca.shape
df_ml_pca

,PC1,PC2,PC3
0,1.281844,0.437146,0.144146
1,-2.212411,1.472249,-1.248292
2,-2.555092,1.409448,-1.987670
3,-0.689044,2.645013,-3.576780
4,-3.223547,0.575506,4.576195
...,...,...,...
4913,4.464401,-1.414774,0.720140
4914,13.966180,1.510254,11.265780
4915,4.360017,1.264705,-0.210211
4916,4.907239,1.156028,0.473722


## Using the elbow curve to find the best number of clusters

In [56]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_ml_pca)
    inertia.append(km.inertia_)

elbow_data = {"k" : k , "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k",y="inertia",xticks=k,title="Elbow Curve")

:Curve   [k]   (inertia)

In [58]:
# Initialize the K-Means model.
k = 4
model = KMeans(n_clusters=k,random_state = 0)

# Fit the model
model.fit(df_ml_pca)

# Predict clusters
predictions = model.predict(df_ml_pca)
predictions

array([3, 1, 1, ..., 3, 3, 1])

In [59]:
clustered_df =pd.concat([ml_df,df_ml_pca], axis=1)
clustered_df["Class"] = model.labels_
clustered_df

,MLSNumber,Orig_List_Price,Zip_Code,Subdivision,School_District,HOA_YN,Structure_Type,Acres,InteriorSqFt,AboveGradeSqFt,...,LotDepth,LotSqFt,LotAcres,YearBuilt,Sold_Price,Days_on_Market,PC1,PC2,PC3,Class
0,DENC518086,330000.0,19808,SHERWOOD PARK II,Red Clay Consolidated,Yes,Detached,0.26,2099.0,2099.0,...,133.5,11326,0.26,1958,335000.0,5.0,1.281844,0.437146,0.144146,3
1,DENC518982,215500.0,19808,PEPPER RIDGE,Red Clay Consolidated,No,Interior Row/Townhouse,0.06,1927.0,1575.0,...,126.2,2614,0.06,1976,200000.0,47.0,-2.212411,1.472249,-1.248292,1
2,DENC512992,200000.0,19808,WOODMILL,Red Clay Consolidated,No,Interior Row/Townhouse,0.05,1150.0,1150.0,...,105.0,2178,0.05,1985,200000.0,15.0,-2.555092,1.409448,-1.987670,1
3,DENC512104,219900.0,19808,FAIRWAY FALLS,Red Clay Consolidated,Yes,Interior Row/Townhouse,0.05,1425.0,1425.0,...,125.0,2178,0.05,1985,200000.0,11.0,-0.689044,2.645013,-3.576780,1
4,DENC506356,195000.0,19808,WASHINGTON HEIGHTS,Red Clay Consolidated,No,Detached,0.14,0.0,0.0,...,118.0,6098,0.14,1954,201000.0,5.0,-3.223547,0.575506,4.576195,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,DENC520114,595000.0,19707,CANTERBURY HILLS,Red Clay Consolidated,Yes,Detached,1.03,3045.0,3045.0,...,0.0,44866,1.03,1958,595000.0,5.0,4.464401,-1.414774,0.720140,3
4914,DENC520392,639000.0,19707,SNUFF MILL,Red Clay Consolidated,Yes,Detached,0.75,3811.0,3175.0,...,300.0,32670,0.75,1973,595000.0,25.0,13.966180,1.510254,11.265780,3
4915,DENC2014038,529900.0,19707,STENNING WOODS,Red Clay Consolidated,No,Detached,0.39,3191.0,2850.0,...,145.0,16988,0.39,1989,600000.0,7.0,4.360017,1.264705,-0.210211,3
4916,DENC2005484,620000.0,19707,QUAKER LEA EAST,Red Clay Consolidated,No,Detached,0.54,3425.0,3425.0,...,190.0,23522,0.54,1992,600000.0,25.0,4.907239,1.156028,0.473722,3


## Visualization

In [74]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    hover_name="MLSNumber",
    hover_data=["Days_on_Market","Orig_List_Price","Sold_Price","Zip_Code"],
    width = 1000,
    height= 1000
    
)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()

In [93]:
labels = clustered_df["MLSNumber"]
labels['color'] = "Sold Price"
fig = px.scatter_matrix(
    clustered_df[["PC1","PC2","PC3"]],
    labels=labels,
    color=clustered_df.Sold_Price
)
fig.update_traces(diagonal_visible=False)
fig.show()

C:\Users\akari\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Check class correlation 

In [102]:
scaler = MinMaxScaler().fit_transform(clustered_df[["Orig_List_Price","Sold_Price"]])
scaler

array([[0.05057062, 0.3375    ],
       [0.0201112 , 0.        ],
       [0.01598787, 0.        ],
       ...,
       [0.10374824, 1.        ],
       [0.12771674, 1.        ],
       [0.05589104, 0.375     ]])

In [108]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(clustered_df, columns=["Days_on_Market","Sold_Price"])

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["MLSNumber"] = clustered_df["MLSNumber"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df["Class"]

plot_df.head(10)

,Days_on_Market,Sold_Price,MLSNumber,Class
0,5.0,335000.0,DENC518086,3
1,47.0,200000.0,DENC518982,1
2,15.0,200000.0,DENC512992,1
3,11.0,200000.0,DENC512104,1
4,5.0,201000.0,DENC506356,0
5,1.0,205000.0,DENC523094,1
6,47.0,205000.0,DENC501750,1
7,7.0,205900.0,DENC505216,0
8,8.0,208000.0,DENC500680,2
9,6.0,208700.0,DENC521992,1


In [109]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="Days_on_Market",y="Sold_Price",by="Class",hover=True,hover_cols="MLSNumber")

:NdOverlay   [Class]
   :Scatter   [Days_on_Market]   (Sold_Price,MLSNumber)

In [ ]:
#TODO: TRY OTHER IMPLEMENTATIONS FOR GRAPH ON TOP ^^^
#TODO: Do Group By with class to find underlying information